In [1]:
from typing import List
import os

from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain_core.pydantic_v1 import BaseModel

In [2]:
from ai_bot_libs.ai_bot import (
    remove_special_chars,
    get_cl_llm,
    get_bedrock_embeddings,
    case_study_search,
    crawl_webpages_vector,
    remove_tags,
    clean_crawled_text_pages,
    google_search,
)

# Bedrock LLM object
cl_llm = get_cl_llm()
# Bedrock Embeddings object
embedding = get_bedrock_embeddings()

# aws cdk documentation
website = "https://docs.aws.amazon.com/cdk/api/v2/python/"

/Users/artemkobrin/repositories/nl/mistral_agent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# run only once to get the embeddings of the website
# website_qdrant = crawl_webpages_vector(
#                 website,
#                 collection_name="aws_cdk",
#                 embedding=embedding,
#                 num_pages=10,
#     )

In [4]:
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_URL=os.getenv("QDRANT_URL")
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

# Qdrant Remote with Use Cases
qdrant_aws_cdk_remote = Qdrant(
    client=qdrant_client,
    collection_name="aws_cdk", 
    embeddings=embedding,
)

retriever = qdrant_aws_cdk_remote.as_retriever(
    search_kwargs={'k': 10},
)

In [5]:
retriever.get_relevant_documents("python aws cdk aws vpc")

concatenated_content = ""
for doc in retriever.get_relevant_documents("python aws cdk aws vpc"):
    concatenated_content += doc.page_content + "\n"

In [6]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""Generate AWS CDK python code to deploy AWS resources with the following properties. 
    You should follow the best practices and use the latest version of the AWS CDK.
    Resources and properties:
    {task}

    Documentation: 
    {context}
""",
    input_variables=["task", "context"], 
)

llm_chain = LLMChain(
    llm=cl_llm,
    prompt=prompt,
)
llm_chain.run(task="VPC with 2 subnets in different availability zones", context=concatenated_content)


/Users/artemkobrin/repositories/nl/mistral_agent/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Here\'s an example AWS CDK Python code to create a VPC with 2 subnets in different availability zones:\n\n```python\nfrom aws_cdk import (\n    Stack,\n    aws_ec2 as ec2,\n    App\n)\nfrom constructs import Construct\n\nclass VpcStack(Stack):\n\n    def __init__(self, scope: Construct, construct_id: str, **kwargs) -> None:\n        super().__init__(scope, construct_id, **kwargs)\n\n        # Create a VPC\n        vpc = ec2.Vpc(\n            self, "MyVpc",\n            max_azs=2,  # Create subnets in 2 availability zones\n            cidr="10.0.0.0/16",\n            subnet_configuration=[\n                ec2.SubnetConfiguration(\n                    name="Public",\n                    subnet_type=ec2.SubnetType.PUBLIC,\n                    cidr_mask=24\n                ),\n                ec2.SubnetConfiguration(\n                    name="Private",\n                    subnet_type=ec2.SubnetType.PRIVATE_WITH_NAT,\n                    cidr_mask=24\n                )\n            ],\n